In [ ]:
# ==========================================
# GAN anomaly detection (creditcard.csv) with manual/F1/Precision threshold
# - Stabilization: BCEWithLogitsLoss, label smoothing, input noise, Adam betas(0.5,0.999)
# - Splits: TrainN=20k normal, VAL 250/250, TEST Balanced 200/200, TEST Imbalanced 10k/200
# ==========================================
import os, random
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    precision_recall_fscore_support, precision_recall_curve
)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# ------------------------------
# 0) Seeds & device
# ------------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ------------------------------
# 1) Load
# ------------------------------
csv_path = "../creditcard.csv"
df = pd.read_csv(csv_path)
assert "Class" in df.columns
X_df = df.drop(columns=["Class"])
y = df["Class"].astype(int).to_numpy()

# ------------------------------
# 2) Splits (no overlap)
# ------------------------------
normal_idx = np.where(y==0)[0]; anom_idx = np.where(y==1)[0]
rng = np.random.default_rng(SEED); rng.shuffle(normal_idx); rng.shuffle(anom_idx)

TR_N, VAL_N, VAL_A, TESTB_N, TESTB_A, TESTI_N = 20000, 250, 250, 200, 200, 10000
assert len(anom_idx) >= (VAL_A + TESTB_A), "Không đủ anomaly."

max_train_normal = len(normal_idx) - (VAL_N + TESTB_N + TESTI_N)
if max_train_normal < 1000:
    TESTI_N = max(2000, len(normal_idx) - (VAL_N + TESTB_N + 1000))
    max_train_normal = len(normal_idx) - (VAL_N + TESTB_N + TESTI_N)
TRAIN_N = max(5000, min(TR_N, max_train_normal))

ptr_n=0; ptr_a=0
trn_n = normal_idx[ptr_n:ptr_n+TRAIN_N]; ptr_n+=TRAIN_N
val_n = normal_idx[ptr_n:ptr_n+VAL_N];   ptr_n+=VAL_N
tstb_n= normal_idx[ptr_n:ptr_n+TESTB_N]; ptr_n+=TESTB_N
tsti_n= normal_idx[ptr_n:ptr_n+TESTI_N]; ptr_n+=TESTI_N

val_a = anom_idx[ptr_a:ptr_a+VAL_A]; ptr_a+=VAL_A
tstb_a= anom_idx[ptr_a:ptr_a+TESTB_A]; ptr_a+=TESTB_A
tsti_a= tstb_a

def take(idxs): return X_df.iloc[idxs].to_numpy().astype(np.float32), y[idxs]
X_tr_n, _ = take(trn_n)

X_val = np.vstack([X_df.iloc[val_n].to_numpy(), X_df.iloc[val_a].to_numpy()]).astype(np.float32)
y_val = np.hstack([np.zeros(len(val_n), dtype=int), np.ones(len(val_a), dtype=int)])

X_tstb = np.vstack([X_df.iloc[tstb_n].to_numpy(), X_df.iloc[tstb_a].to_numpy()]).astype(np.float32)
y_tstb = np.hstack([np.zeros(len(tstb_n), dtype=int), np.ones(len(tstb_a), dtype=int)])

X_tsti = np.vstack([X_df.iloc[tsti_n].to_numpy(), X_df.iloc[tsti_a].to_numpy()]).astype(np.float32)
y_tsti = np.hstack([np.zeros(len(tsti_n), dtype=int), np.ones(len(tsti_a), dtype=int)])

print(f"TrainN={len(trn_n)}, Val={len(val_n)}/{len(val_a)}, TestB={len(tstb_n)}/{len(tstb_a)}, TestI={len(tsti_n)}/{len(tsti_a)}")

# ------------------------------
# 3) Scale (fit on train normal)
# ------------------------------
scaler = StandardScaler().fit(X_tr_n)
def z(x): return scaler.transform(x).astype(np.float32)
X_tr_n = z(X_tr_n); X_val = z(X_val); X_tstb = z(X_tstb); X_tsti = z(X_tsti)

# ------------------------------
# 4) GAN
# ------------------------------
class Generator(nn.Module):
    def __init__(self, latent_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, out_dim)
        )
    def forward(self, z): return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 1)  # logits
        )
    def forward(self, x): return self.net(x)

input_dim = X_tr_n.shape[1]; latent_dim = 32
G = Generator(latent_dim, input_dim).to(device)
D = Discriminator(input_dim).to(device)

bce_logits = nn.BCEWithLogitsLoss()
opt_g = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5,0.999))
opt_d = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5,0.999))

bs, epochs = 256, 60
loader = DataLoader(TensorDataset(torch.from_numpy(X_tr_n)), batch_size=bs, shuffle=True, drop_last=False)

for ep in range(1, epochs+1):
    G.train(); D.train()
    dsum=gsum=0.0; steps=0
    for (xb,) in loader:
        xb = xb.to(device)
        bsz = xb.size(0)
        real_noisy = xb + 0.01 * torch.randn_like(xb)

        # Train D
        opt_d.zero_grad()
        z = torch.randn(bsz, latent_dim, device=device)
        fake = G(z).detach()
        real_lbl = torch.full((bsz,1), 0.9, device=device)
        fake_lbl = torch.zeros(bsz,1, device=device)
        d_real = D(real_noisy)
        d_fake = D(fake)
        d_loss = bce_logits(d_real, real_lbl) + bce_logits(d_fake, fake_lbl)
        d_loss.backward(); opt_d.step()

        # Train G
        opt_g.zero_grad()
        z = torch.randn(bsz, latent_dim, device=device)
        gen = G(z)
        g_loss = bce_logits(D(gen), torch.ones(bsz,1,device=device))
        g_loss.backward(); opt_g.step()

        dsum += d_loss.item(); gsum += g_loss.item(); steps += 1
    if ep==1 or ep%5==0:
        print(f"[GAN] Epoch {ep:3d}/{epochs} | D={dsum/steps:.5f} | G={gsum/steps:.5f}")

@torch.no_grad()
def disc_score(x_np: np.ndarray):
    D.eval()
    xt = torch.from_numpy(x_np).to(device)
    logits = D(xt).cpu().numpy().reshape(-1)
    probs  = 1.0/(1.0+np.exp(-logits))
    return 1.0 - probs  # higher => more anomalous

# ------------------------------
# 5) Threshold selection
# ------------------------------
MODE = "f1"           # "manual" | "f1" | "p_at"
THR_MANUAL = 0.60     # dùng khi MODE="manual"
TARGET_P   = 0.60     # dùng khi MODE="p_at"

val_scores = disc_score(X_val)

def best_f1_threshold(y_true, scores, percentiles=np.linspace(50, 99.5, 200)):
    ths = np.percentile(scores, percentiles)
    best_f1, best_thr = -1.0, None
    for t in ths:
        yhat = (scores >= t).astype(int)
        _, _, f1, _ = precision_recall_fscore_support(
            y_true, yhat, labels=[0,1], average=None, zero_division=0
        )
        if f1[1] > best_f1:
            best_f1, best_thr = float(f1[1]), float(t)
    return best_thr, best_f1

def threshold_for_precision(y_true, scores, target_p=0.60):
    p, r, thr = precision_recall_curve(y_true, scores)
    idx = np.where(p[:-1] >= target_p)[0]
    if len(idx) == 0:
        # fallback: dùng percentile 95 nếu không đạt target precision
        return float(np.percentile(scores, 95)), float(p[1] if len(p)>1 else 0.0), float(r[1] if len(r)>1 else 0.0)
    i = idx[0]
    return float(thr[i]), float(p[i]), float(r[i])

if MODE == "manual":
    thr = float(THR_MANUAL)
    pct = (val_scores <= thr).mean() * 100.0
    print(f"\n[VAL] Manual threshold selected: thr={thr:.6f} (~percentile {pct:.2f}%)")
elif MODE == "f1":
    thr, f1v = best_f1_threshold(y_val, val_scores)
    print(f"\n[VAL balanced] Best F1(Class 1)={f1v:.3f} at threshold={thr:.6f}")
else:  # "p_at"
    thr, p_at, r_at = threshold_for_precision(y_val, val_scores, TARGET_P)
    print(f"\n[VAL balanced] Threshold for Precision≥{TARGET_P:.2f}: thr={thr:.6f} (P={p_at:.3f}, R={r_at:.3f})")

# ------------------------------
# 6) Evaluate
# ------------------------------
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

def evaluate(name, X, y, thr):
    s = disc_score(X); yhat = (s >= thr).astype(int)
    print(f"\n===== {name} @thr={thr:.6f} =====")
    print("Confusion Matrix:\n", confusion_matrix(y, yhat))
    print("\nClassification Report:\n", classification_report(y, yhat, digits=4))
    print("ROC AUC (scores):", roc_auc_score(y, s))

evaluate("TEST Balanced (200/200)",     X_tstb, y_tstb, thr)
evaluate("TEST Imbalanced (10000/200)", X_tsti, y_tsti, thr)

# ------------------------------
# 7) Quick sweep vài percentile để quan sát
# ------------------------------
cands = [50, 60, 70,80, 90, 92.5, 95, 97.5, 99]
print("\n>>> Quick sweep on percentile-based thresholds (from VAL):")
for p in cands:
    t = float(np.percentile(val_scores, p))
    print(f"\n-- Try thr={t:.6f} (pctl={p}) on TEST Balanced --")
    evaluate("TEST Balanced (200/200)", X_tstb, y_tstb, t)
    print(f"\n-- Try thr={t:.6f} (pctl={p}) on TEST Imbalanced --")
    evaluate("TEST Imbalanced (10000/200)", X_tsti, y_tsti, t)


Device: cpu
TrainN=20000, Val=250/250, TestB=200/200, TestI=10000/200
[GAN] Epoch   1/60 | D=1.20100 | G=0.70563
[GAN] Epoch   5/60 | D=1.04195 | G=1.28119
[GAN] Epoch  10/60 | D=1.12159 | G=1.21620
[GAN] Epoch  15/60 | D=1.20607 | G=1.24549
[GAN] Epoch  20/60 | D=1.09469 | G=1.27483
[GAN] Epoch  25/60 | D=1.15619 | G=1.30272
[GAN] Epoch  30/60 | D=1.10116 | G=1.13824
[GAN] Epoch  35/60 | D=1.14603 | G=1.32037
[GAN] Epoch  40/60 | D=1.07364 | G=1.40558
[GAN] Epoch  45/60 | D=1.02368 | G=1.33516
[GAN] Epoch  50/60 | D=1.01686 | G=1.34203
[GAN] Epoch  55/60 | D=0.95335 | G=1.42906
[GAN] Epoch  60/60 | D=0.90034 | G=1.65828

[VAL balanced] Best F1(Class 1)=0.748 at threshold=0.692722

===== TEST Balanced (200/200) @thr=0.692722 =====
Confusion Matrix:
 [[167  33]
 [ 75 125]]

Classification Report:
               precision    recall  f1-score   support

           0     0.6901    0.8350    0.7557       200
           1     0.7911    0.6250    0.6983       200

    accuracy                